In [1]:
%matplotlib 
from matplotlib import pyplot as plt
import numpy as np
import matplotlib 
from PIL import Image
import PIL
import math
from random import random
from random import seed
from random import randint
import time
from numba import jit
import skimage

Using matplotlib backend: Qt5Agg


In [69]:
def Laplacian(A,deltax,deltat):

    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions           

    
    Ax1 = np.roll(A,1,axis=0)
    Ax2 = np.roll(A,2,axis=0)
    Ax_1 = np.roll(A,-1,axis=0)
    Ax_2 = np.roll(A,-2,axis=0)
    
    Ay1 = np.roll(A,1,axis=1)
    Ay2 = np.roll(A,2,axis=1)
    Ay_1 = np.roll(A,-1,axis=1)
    Ay_2 = np.roll(A,-2,axis=1)
    
    P = (-Ax2 + 16*Ax1 + 16*Ax_1 - Ax_2 - Ay2 + 16*Ay1 + 16*Ay_1 - Ay_2 - 60*A)/(12*pow(deltax,2))
    
  
    return P


def div_potential_1(A1,A2,omega,eta):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A2*(A1*A2-pow(eta,2))/2
    return pot

def div_potential_2(A1,A2,omega,eta):
    lam = 2*pow(math.pi,2)/pow(omega,2)
    pot = lam*A1*(A1*A2-pow(eta,2))/2
    return pot
    
def function_1(A1,A2,omega,eta,deltax,deltat):
    F = np.subtract(Laplacian(A1,deltax,deltat),div_potential_1(A1,A2,omega,eta))
    return F

def function_2(A1,A2,omega,eta,deltax,deltat):
    F = np.subtract(Laplacian(A2,deltax,deltat),div_potential_2(A1,A2,omega,eta))
    return F

def update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat):
    F1 = function_1(A1,A2,omega,eta,deltax,deltat)
    F2 = function_2(A1,A2,omega,eta,deltax,deltat)
    
    B1 = A1
    
    A1 = A1 + deltat*(vel1 + 0.5*deltat*function_1(A1,A2,omega,eta,deltax,deltat)) 
    A2 = A2 + deltat*(vel2 + 0.5*deltat*function_2(B1,A2,omega,eta,deltax,deltat)) 
    
    vel1 = vel1 + deltat/2*(F1 + function_1(A1,A2,omega,eta,deltax,deltat)) 
    vel2 = vel2 + deltat/2*(F2 + function_2(A1,A2,omega,eta,deltax,deltat))
    return A1,A2,vel1,vel2
    
#change the values accordlingly    
def wolff(N,t,A,vel,omega,eta,deltax,deltat):

    A1 = A
    A2 = np.matrix.conjugate(A)
    vel1 = vel
    vel2 = vel
    
    plt.figure(0)
    plt.ion()
    plt.show()
    plt.gray()

    
    for i in np.arange(t/deltat):
        print(A1[1,1])
        A1,A2,vel1,vel2 = update(N,A1,A2,vel1,vel2,omega,eta,deltax,deltat)
        absA = np.absolute(A1)
        
        if i == 0:
            plt.imshow(absA, vmin= 0, vmax=1.5*round(eta),cmap = 'Greys', interpolation = 'none')
            plt.colorbar()
        
        if i in range(50) or i % pow(2,n-1) == 0 and i != 0:
            plt.imshow(absA, vmin= 0, vmax=1.5*round(eta))
            plt.draw()
            plt.pause(0.001)
            plt.title('time= '+str(round((i*deltat),1))+'')

    return A1


In [70]:
n = 5
deltax = 1
deltat = 0.1
eta = 1
N = pow(2,n)
t = 0.25*N*deltax/deltat
vel = np.zeros((N,N))
#mu, sigma = 0, 0.1 # mean and standard deviation
omega = 5

#A = np.random.normal(mu, sigma, size=(N, N))
A = np.random.randn(N, 2*N).view(np.complex128)   

A = wolff(N,t,A,vel,omega,eta,deltax,deltat)

(-0.2765586723373841-0.04591297027393348j)
(-0.2735471346609353-0.043671365981696006j)
(-0.26476873528950906-0.037254778331001j)
(-0.250949375275697-0.027557119971646442j)
(-0.23316533102668788-0.01596843802880074j)
(-0.21267201025158852-0.004234564059811043j)
(-0.19071414636591716+0.005721416429592023j)
(-0.16839738899203802+0.01199964338708454j)
(-0.14659748100128545+0.012927382276423332j)
(-0.1258690909778577+0.0072470571408527916j)
(-0.1063896898408083-0.005729982862220454j)
(-0.08796004975415361-0.026019839632907132j)
(-0.07005283985067683-0.052924443943478484j)
(-0.05195209971706714-0.08505855114361818j)
(-0.03292716581037339-0.12044588701976094j)
(-0.012415849481389664-0.1566785292598165j)
(0.009791993930013274-0.19112734198878326j)
(0.03340406224501813-0.22118575688365782j)
(0.05752392518360501-0.24452495729227178j)
(0.08061233831549582-0.2593360431202202j)
(0.10051393482741153-0.2645343863530093j)
(0.11448866126628457-0.25990332075225814j)
(0.11932316212995653-0.24615854170996

<ipython-input-69-06aa408d24c3>:24: RuntimeWarning: overflow encountered in multiply
  pot = np.absolute(lam*A2*(A1*A2-pow(eta,2))/2)
<ipython-input-69-06aa408d24c3>:24: RuntimeWarning: invalid value encountered in true_divide
  pot = np.absolute(lam*A2*(A1*A2-pow(eta,2))/2)
<ipython-input-69-06aa408d24c3>:49: RuntimeWarning: invalid value encountered in multiply
  vel1 = vel1 + deltat/2*(F1 + function_1(A1,A2,omega,eta,deltax,deltat))
<ipython-input-69-06aa408d24c3>:29: RuntimeWarning: overflow encountered in multiply
  pot = np.absolute(lam*A1*(A1*A2-pow(eta,2))/2)
<ipython-input-69-06aa408d24c3>:29: RuntimeWarning: invalid value encountered in true_divide
  pot = np.absolute(lam*A1*(A1*A2-pow(eta,2))/2)
<ipython-input-69-06aa408d24c3>:50: RuntimeWarning: invalid value encountered in multiply
  vel2 = vel2 + deltat/2*(F2 + function_2(A1,A2,omega,eta,deltax,deltat))
<ipython-input-69-06aa408d24c3>:46: RuntimeWarning: invalid value encountered in multiply
  A1 = A1 + deltat*(vel1 + 0.5

(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)

(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)
(nan+nanj)


In [71]:
A = np.random.randn(N, 2*N).view(np.complex128)
print(A.shape)
print(A)

(32, 32)
[[ 3.48239222e-01-0.086864j   -9.72252723e-01-0.90861984j
  -9.86784924e-01+0.97497727j ... -1.84952964e-01-0.08199657j
  -4.78528574e-01+0.46391929j -6.29576326e-01-0.95245571j]
 [ 6.07910966e-02+1.22451176j  2.02692658e-01+0.0878878j
  -1.66019994e+00+0.51922098j ... -4.24962611e-01-0.70776862j
   7.86196394e-01-0.9334488j   1.89286466e+00-0.7148575j ]
 [ 2.12928392e+00-2.02406412j  1.04011512e+00-0.06735334j
   2.84498947e-02-0.56397258j ...  1.55787639e+00+0.95667097j
   8.92650635e-01+0.29237422j -1.20608478e+00+0.3082954j ]
 ...
 [-7.64544538e-01-1.00316074j  9.00028764e-01+1.14559068j
  -2.44382318e+00+0.29929898j ...  6.84635010e-01+0.25969286j
   8.58836771e-01+0.64639842j  2.67265820e-01-0.35388006j]
 [ 4.44898260e-02+0.37043432j  1.33457644e+00-0.16176988j
   3.54520446e-01-0.6290447j  ... -4.19417146e-01+0.04022727j
  -8.52697692e-01+0.2355385j  -3.32598602e-01+0.1887967j ]
 [ 1.05325674e+00+0.43118733j  1.33155643e+00-0.89706393j
   2.21090672e-03-0.8275574j  ... 

In [ ]:
@jit(nopython = True)
    
def Laplacian(A,deltax,deltat):

    h = len(A)
    #CFL condition delta x => sqrt(D) delta t, where D is the number of dimensions            
    P = np.zeros((h,h))
    
    for i in range(0, h):
        for j in range(0,h):
            P[i,j] = (-A[(i+2)%h,j] + 16*A[(i+1)%h,j] + 16*A[(i-1)%h,j] -A[(i-2)%h,j] \
                 -A[i,(j+2)%h] + 16*A[i,(j+1)%h]  + 16*A[i,(j-1)%h] -A[i,(j-2)%h] - 60*A[i,j])/(12*pow(deltax,2))

  
    return P 